In [51]:
import pandas as pd
import datetime as dt
import numpy as np

In [52]:
#from ydata_profiling import ProfileReport

In [53]:
from config import sheet_id, sheet_name

In [54]:
import os
os.getcwd()

'c:\\Users\\jimmy\\Documents\\Data Science Folder\\basecamp_salary_dashboard'

In [55]:
# import sheet_id and sheet_name from a local config file
sheet_id = sheet_id
sheet_name = sheet_name
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
# replace whitespace in the URL
url = url.replace(" ", "%20")

In [56]:
# specify columns to read in (all but the "Company Name" column)
# parse_dates argument tells us which column is a datetime field
df_raw = pd.read_csv(url, usecols = lambda x: x != "Company Name", parse_dates = ['Submitted At'])

In [57]:
df_raw.head()

,I am a:,Industry,Job Title,Location,Flexibility,# Years,Pay,I charge per:,Bonuses/ Commissions,Equity,...,Field of work,City,Government role?,Parental leave,Pronouns,Race/Ethnicity,I identify as part of the following communities,Submitted At,Token,Unnamed: 27
0,Full-Time Employee,Active Lifestyle,North America Marketing Manager,Oregon,Hybrid/Work From Home,15+,95000,/Year,10000.0,False,...,Marketing & Branding,"Portland, OR",Nope,6 months,She/Her,NaN,"LGBTQIA2S+, Woman",2022-10-29 22:59:19,4uhvwonebzd2xry77gpme4uhvwn6pmnd,NaN
1,Full-Time Employee,Running,PR Director,Colorado,Fully Remote,8,165000,/Year,NaN,True,...,PR & Communications,Denver,Nope,3 months,He/Him,Latino/Hispanic,LGBTQIA2S+,2022-10-29 22:56:19,9c19jk3coesvcdu9c19px0ltwrzj1cbn,NaN
2,Full-Time Employee,Travel/Tourism,Vice President of Brand Marketing,California,Fully Remote,5,150000,/Year,4000.0,False,...,Hospitality & Adventure Tourism,San Francisco,Nope,2 months,She/Her,NaN,Woman,2022-10-29 22:23:42,7bnz7dq3fbcyg3xf0vzx97bnz7pn3bij,NaN
3,Full-Time Employee,Bike,Store Manager,Pennsylvania,In-Office,6,66000,/Year,15000.0,True,...,Retail,Philadelphia,Nope,2 months,He/Him,NaN,NaN,2022-10-29 22:14:55,mzpxp2psypsbtt1hcespgmzpxp2p7arl,NaN
4,Full-Time Employee,Bike,People Ops,Oregon,Fully Remote,4,80000,/Year,0.0,False,...,Human Resources (seems like it should be an op...,"Bend, OR",Nope,1 months,She/Her,NaN,"50+, LGBTQIA2S+, Woman",2022-10-29 21:58:27,8xv58qt6gkciedqai2v8xv58qt69tmh2,NaN


In [58]:
len(df_raw["Pay"])

928

In [59]:
df = df_raw.copy(deep = True)

In [60]:
df.columns

Index(['I am a:', 'Industry', 'Job Title', 'Location', 'Flexibility',
       '# Years', 'Pay', 'I charge per:', 'Bonuses/ Commissions', 'Equity',
       'PTO', 'Training/Education', 'Add'l Qs', 'Benefits', 'OTJ Experience',
       'Starting Pay', 'Specialty expertise', 'Field of work', 'City',
       'Government role?', 'Parental leave ', 'Pronouns', 'Race/Ethnicity',
       'I identify as part of the following communities', 'Submitted At',
       'Token', 'Unnamed: 27'],
      dtype='object')

In [61]:
# general syntax of loc
# df.loc['row_label', 'column_label']

# If there are multiple labels, they should be specified inside lists:
# df.loc[['row_1', 'row_2'], ['column_1', 'column_2']]


In [62]:
# specify which columns to keep
columns_wanted = ['I am a:', 'Industry', 'Location', 'Flexibility', '# Years', 'Pay', 'I charge per:', 'Field of work', 'Submitted At', 'Pronouns']

In [63]:
# create a subset of the DataFrame with specified columns
df_subset = df.loc[:, columns_wanted]

In [64]:
df_subset.head()

,I am a:,Industry,Location,Flexibility,# Years,Pay,I charge per:,Field of work,Submitted At,Pronouns
0,Full-Time Employee,Active Lifestyle,Oregon,Hybrid/Work From Home,15+,95000,/Year,Marketing & Branding,2022-10-29 22:59:19,She/Her
1,Full-Time Employee,Running,Colorado,Fully Remote,8,165000,/Year,PR & Communications,2022-10-29 22:56:19,He/Him
2,Full-Time Employee,Travel/Tourism,California,Fully Remote,5,150000,/Year,Hospitality & Adventure Tourism,2022-10-29 22:23:42,She/Her
3,Full-Time Employee,Bike,Pennsylvania,In-Office,6,66000,/Year,Retail,2022-10-29 22:14:55,He/Him
4,Full-Time Employee,Bike,Oregon,Fully Remote,4,80000,/Year,Human Resources (seems like it should be an op...,2022-10-29 21:58:27,She/Her


In [65]:
df_subset["I am a:"].value_counts()

Full-Time Employee          793
Contract/Seasonal Worker     88
Part-Time Employee           41
Intern                        5
Board Member                  1
Name: I am a:, dtype: int64

In [66]:
df_subset["I charge per:"].value_counts()

/Year       687
/Hour       202
/Month       26
/Project     13
Name: I charge per:, dtype: int64

In [67]:
# check datatypes
df_subset.dtypes

I am a:                  object
Industry                 object
Location                 object
Flexibility              object
# Years                  object
Pay                       int64
I charge per:            object
Field of work            object
Submitted At     datetime64[ns]
Pronouns                 object
dtype: object

In [68]:
# subset only responses with 'I charge per: ' '/Year'
df_subset = df_subset.loc[df_subset['I charge per:'] == '/Year']

In [69]:
# Extract the Year from the Submitted At column
df_subset['Year'] = df_subset['Submitted At'].dt.strftime('%Y')

In [70]:
# Rename columns
df_subset.rename(columns = {"# Years": "Years of Experience"}, inplace = True)

In [71]:
df_subset['Years of Experience'].value_counts()

10+                 100
2                    90
1                    72
Less than a year     71
3                    65
4                    63
5                    57
15+                  39
7                    32
6                    29
8                    24
9                    20
20+                  18
25+                   6
30+                   1
Name: Years of Experience, dtype: int64

In [72]:
# Create categories for "Years of Experience"
df_subset['Years of Exp.'] = np.select(
    [
        df_subset["Years of Experience"] == 'Less than a year',
        df_subset["Years of Experience"] == '1',
        df_subset["Years of Experience"] == '2',
        df_subset["Years of Experience"] == '3',
        df_subset["Years of Experience"] == '4',
        df_subset["Years of Experience"] == '5',
        df_subset["Years of Experience"] == '6',
        df_subset["Years of Experience"] == '7',
        df_subset["Years of Experience"] == '8',
        df_subset["Years of Experience"] == '9',
        df_subset["Years of Experience"] == '10+',
        df_subset["Years of Experience"] == '15+',
        df_subset["Years of Experience"] == '20+',
        df_subset["Years of Experience"] == '25+',
        df_subset["Years of Experience"] == '23+' 
    ],
    [
        '0-3 years',
        '0-3 years',
        '0-3 years',
        '0-3 years',
        '4-6 years',
        '4-6 years',
        '4-6 years',
        '7-9 years',
        '7-9 years',
        '7-9 years',
        '10+ years',
        '10+ years',
        '10+ years',
        '10+ years',
        '10+ years'
    ],
    default = 'No Data'
)

In [73]:
df_subset["Pronouns"].value_counts()

She/Her      435
He/Him       217
They/Them      7
she/they       2
She/They       1
She/they       1
Name: Pronouns, dtype: int64

In [74]:
# Create categories for "Pronouns"
df_subset['Pronouns'] = np.select(
    [
        df_subset["Pronouns"] == 'She/Her',
        df_subset["Pronouns"] == 'He/Him',
        df_subset["Pronouns"] == 'They/Them',
        df_subset["Pronouns"] == 'she/they',
        df_subset["Pronouns"] == 'She/They',
        df_subset["Pronouns"] == 'She/they'
    ],
    [
        'She/Her',
        'He/Him',
        'They/Them',
        'She/They',
        'She/They',
        'She/They'
    ],
    default = 'No Response'
)

In [84]:
df_subset["Pronouns"].value_counts()

She/Her        431
He/Him         217
No Response     24
They/Them        7
She/They         4
Name: Pronouns, dtype: int64

In [75]:
df_subset.head(10)

,I am a:,Industry,Location,Flexibility,Years of Experience,Pay,I charge per:,Field of work,Submitted At,Pronouns,Year,Years of Exp.
0,Full-Time Employee,Active Lifestyle,Oregon,Hybrid/Work From Home,15+,95000,/Year,Marketing & Branding,2022-10-29 22:59:19,She/Her,2022,10+ years
1,Full-Time Employee,Running,Colorado,Fully Remote,8,165000,/Year,PR & Communications,2022-10-29 22:56:19,He/Him,2022,7-9 years
2,Full-Time Employee,Travel/Tourism,California,Fully Remote,5,150000,/Year,Hospitality & Adventure Tourism,2022-10-29 22:23:42,She/Her,2022,4-6 years
3,Full-Time Employee,Bike,Pennsylvania,In-Office,6,66000,/Year,Retail,2022-10-29 22:14:55,He/Him,2022,4-6 years
4,Full-Time Employee,Bike,Oregon,Fully Remote,4,80000,/Year,Human Resources (seems like it should be an op...,2022-10-29 21:58:27,She/Her,2022,4-6 years
5,Full-Time Employee,Climb,New York,In-Office,9,62000,/Year,Administrative/Virtual Assistance,2022-10-29 21:16:49,She/They,2022,7-9 years
6,Contract/Seasonal Worker,Non-Profit,Kentucky,Hybrid/Work From Home,1,81000,/Year,Partnerships & Community,2022-10-29 20:52:11,He/Him,2022,0-3 years
7,Full-Time Employee,Bike,Colorado,Hybrid/Work From Home,3,85000,/Year,Marketing & Branding,2022-10-29 20:28:26,She/Her,2022,0-3 years
8,Full-Time Employee,Ski/Snow,Maine,In-Office,15+,75000,/Year,Marketing & Branding,2022-10-29 20:11:12,He/Him,2022,10+ years
9,Full-Time Employee,Conservation,Georgia,Hybrid/Work From Home,1,117692,/Year,PR & Communications,2022-10-29 19:30:55,He/Him,2022,0-3 years


In [82]:
# remove outliers with pay less than $100 per year OR more than $500,000 per year
index_sal_100_500k = df_subset[(df_subset['Pay'] <= 100) | (df_subset['Pay'] >= 500000)].index
df_subset.drop(index_sal_100_500k , inplace=True)

In [91]:
df_subset.head()

,I am a:,Industry,Location,Flexibility,Years of Experience,Pay,I charge per:,Field of work,Submitted At,Pronouns,Year,Years of Exp.
Index,,,,,,,,,,,,
0,Full-Time Employee,Active Lifestyle,Oregon,Hybrid/Work From Home,15+,95000,/Year,Marketing & Branding,2022-10-29 22:59:19,She/Her,2022,10+ years
1,Full-Time Employee,Running,Colorado,Fully Remote,8,165000,/Year,PR & Communications,2022-10-29 22:56:19,He/Him,2022,7-9 years
2,Full-Time Employee,Travel/Tourism,California,Fully Remote,5,150000,/Year,Hospitality & Adventure Tourism,2022-10-29 22:23:42,She/Her,2022,4-6 years
3,Full-Time Employee,Bike,Pennsylvania,In-Office,6,66000,/Year,Retail,2022-10-29 22:14:55,He/Him,2022,4-6 years
4,Full-Time Employee,Bike,Oregon,Fully Remote,4,80000,/Year,Human Resources (seems like it should be an op...,2022-10-29 21:58:27,She/Her,2022,4-6 years


In [90]:
df_subset.index.name = 'Index'

In [101]:
industries = df_subset.Industry.unique()

In [102]:
for i in industries:
    print(i)

Active Lifestyle
Running
Travel/Tourism
Bike
Climb
Non-Profit
Ski/Snow
Conservation
Outdoor (Multi-Industry Apparel/ Gear/Services)
Education
Tech
Camp Jobs
Rafting
Camp/Overland/Vanlife
Higher Education - Outdoor Programs
Hunt/Fish
Outdoor Production (docos, commercials, creative work)
addiction support
Outdoor Education
Outdoor Adjacent
Outdoor Media
Healthcare
Parks and Recreation
Big Data ,cloud computing, medical software
Dealership (surf boats, snowmobile, atv/side by sides)
Outdoor Gear Designer
RV
Media (outdoor/active lifestyle)
Surf/Paddle
Sales - Golf
Adventure Park
Higher education trails program
CPG/Food/Sports Nutrition
Wellness
Utility Consultant
State Government
Government
Advertising
Wildlife
Outdoor Recreation Non-profit
Outdoor Education (College)
Education/higher ed
Search and Rescue
Marketing
media - sports
Clinical Trials
Insurance
Environmental Health & Safety
Public Horticulture
Outdoor & Rec-Heavy Town Position
Food
Local Government Parks & Recreation Design De

In [99]:
len(df_subset.Industry.unique())

60

In [93]:
df_subset.to_csv("salary_data.csv")

In [92]:
df_subset.Pay.describe()

count       683.000000
mean      75841.102489
std       35824.805200
min        1500.000000
25%       53000.000000
50%       68000.000000
75%       88250.000000
max      403950.000000
Name: Pay, dtype: float64

In [79]:
df_subset.Year.value_counts()

2022    576
2023    111
Name: Year, dtype: int64

In [80]:
# profile = ProfileReport(df_subset, title = "Profiling Report")

In [81]:
# profile.to_file("data_quality_report.html")